# Objective
Is there a relationship between wager amounts and win/loss records?

In [ ]:
from db import fetch
from tqdm import tqdm

In [ ]:
query = """
select userid, contestid, matchmaking_settings_amounts.value, contests.winner, contests.__updatetime

from matchmaking

inner join

matchmaking_settings_amounts

on matchmaking_settings_amounts.matchmaking_settings_id = matchmaking.id

inner join

contests

on contests.id = matchmaking.contestid

LIMIT 10000
"""

In [ ]:
df = fetch(query)
df.shape

In [ ]:
df.head()

In [ ]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    u = row['userid'] 
    v = row['value']
    t = row['__updatetime']
    w = row['winner']
    s = row['userid'] == row['winner']
    
    
    if u in lookup.keys():
        if s:
            lookup[u].append((t, "W", v))
        else:
            lookup[u].append((t, "L", v))
    else:
        if s:
            lookup[u] = [(t, "W", v)]
        else:
            lookup[u] = [(t, "L", v)]

In [ ]:
# sort games by timestamp
for k, v in lookup.items():
    lookup[k] = sorted(v, key=lambda x: x[0])

In [ ]:
# remove timestamp
for k, v in lookup.items():
    lookup[k] = list(map(lambda x: (x[1] + x[2]), v))

In [ ]:
for k, v in lookup.items():
    if len(v) > 3:
        print("userid {}: record {}".format(k, v))